<a href="https://colab.research.google.com/github/WilliamHackspeare/chess-environment/blob/main/Chess_Environment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Chess Environment

This project is meant to showcase a playground for human and programmatic chess play, building upon the python-chess and stockfish modules.

